# Example Sensor Data ML Notebook
Examples of humidity and temperature sensor telemetry Machine Learning (ML) using scikit-learn (https://scikit-learn.org/stable/)

In [ ]:
import os
import warnings

import joblib
import matplotlib.pyplot as plt
import pandas as pd
import pymongo
from dotenv import load_dotenv
from mlxtend.plotting import plot_decision_regions
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

warnings.filterwarnings('ignore')
%matplotlib inline

## Load Environment Variables and Establish MongoDB Atlas Connection

In [ ]:
%%time
# load env vars from local .env file
load_dotenv()

# establish db connection
MONGODB_CONN = os.environ.get('MONGODB_CONN')
MONGODB_DB = os.environ.get('MONGODB_DB')
MONGODB_COL = os.environ.get('MONGODB_COL')

client = pymongo.MongoClient(MONGODB_CONN)
db = client[MONGODB_DB]
iot_data = db[MONGODB_COL]

## Define Panda DataFrames for (3) DHT Sensors
Panda DataFrames:
1. DataFrame 1: df1 = rb47a3d9f5
2. DataFrame 2: df2 = rp829c7e0e
3. DataFrame 3: df3 = rp47a3d9f5

In [ ]:
DEVICE_1 = 'rp59adf374'
DEVICE_2 = 'rp829c7e0e'
DEVICE_3 = 'rp47a3d9f5'

### DataFrame 1

In [ ]:
%%time
# aggregation created and exported from MongoDB Compass
# ~72-hour period from 5/11/2019 8:00:00 PM - 5/14/2019 8:00:00 PM
pipeline = [
    {
        '$match': {
            'type': 'DHT22', 
            'device': DEVICE_1, 
            'timestamp': {
                '$gt': 1557619200,
                '$lt': 1557878400
            }
        }
    }, {
        '$project': {
            '_id': 0,
            'device': 1,
            'temperature': '$data.temperature', 
            'humidity': '$data.humidity'
        }
    }
]
aggResult = iot_data.aggregate(pipeline)
df1 = pd.DataFrame(list(aggResult))

# convert celsius to fahrenheit
df1['temperature'] = ((df1.temperature * 1.8) + 32)

# attempt to remove outliers (sensor anomalies)
qt = df1['temperature'].quantile(0.01)
df1 = df1[df1['temperature'] >= qt]
qh = df1['humidity'].quantile(0.01)
df1 = df1[df1['humidity'] >= qh]

### DataFrame 2

In [ ]:
%%time
# aggregation created and exported from MongoDB Compass
# ~72-hour period from 5/11/2019 8:00:00 PM - 5/14/2019 8:00:00 PM
pipeline = [
    {
        '$match': {
            'type': 'DHT22', 
            'device': DEVICE_2, 
            'timestamp': {
                '$gt': 1557619200,
                '$lt': 1557878400
            }
        }
    }, {
        '$project': {
            '_id': 0,
            'device': 1,
            'temperature': '$data.temperature', 
            'humidity': '$data.humidity'
        }
    }
]
aggResult = iot_data.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))

# convert celsius to fahrenheit
df2['temperature'] = ((df2.temperature * 1.8) + 32)

# attempt to remove outliers (sensor anomalies)
qt = df2['temperature'].quantile(0.01)
df2 = df2[df2['temperature'] >= qt]
qh = df2['humidity'].quantile(0.01)
df2 = df2[df2['humidity'] >= qh]

### DataFrame 3

In [ ]:
%%time
# aggregation created and exported from MongoDB Compass
# ~32-hour period
# 2019-05-17 21:58:04.746520996-04:00
# 2019-05-19 07:59:55.743566036-04:00
pipeline = [
    {
        '$match': {
            'type': 'DHT22', 
            'device': DEVICE_3, 
            'timestamp': {
                '$gt': 1558094400,
                '$lt': 1558267200
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'device': 1,
            'temperature': '$data.temperature', 
            'humidity': '$data.humidity'
        }
    }
]
aggResult = iot_data.aggregate(pipeline)
df3 = pd.DataFrame(list(aggResult))

# convert celsius to fahrenheit
df3['temperature'] = ((df3.temperature * 1.8) + 32)

# attempt to remove outliers (sensor anomalies)
qt = df3['temperature'].quantile(0.01)
df3 = df3[df3['temperature'] >= qt]
qh = df3['humidity'].quantile(0.01)
df3 = df3[df3['humidity'] >= qh]

## Prepare Data for Modeling

In [ ]:
# concat the (3) dataframes (different iot devices)
df_devices = [df1, df2, df3]
df_devices = pd.concat(df_devices, sort=False)
df_devices = df_devices.sample(frac=1)

# split dataframe into data and labels numpy.ndarray's
X = df_devices.iloc[:, [1,2]].values
y = df_devices.iloc[:, 0].values

# encode labels with value between 0 and n_classes-1
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

print('df_devices:\n', df_devices.head(5), '\n')
print('X:\n', X[:5], '\n')
print('y:\n', y[:10], '\n')

In [ ]:
# split data into training and testing data
test_size = 0.25
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)

### Model/Data Plotting Function

In [ ]:
def plot_model(clf, title):
    fig = plt.figure(figsize=(11,7))
    fig = plot_decision_regions(X, y, clf=clf, legend=1)
    plt.xlabel('Humidity (%)')
    plt.ylabel('Temperature (°F)')
    plt.title(title)
    plt.legend()
    plt.show()

### Computing Cross-Validation Metrics
<https://scikit-learn.org/stable/modules/cross_validation.html>

In [ ]:
def cross_validation(model):
    model.score(X_test, y_test) 
    scores = cross_val_score(model, X_test, y_test, cv=5)
    return ("Model accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### Make Predictions using Model

In [ ]:
def make_prediction(model, x, y):
    return le.inverse_transform(model.predict([[x, y]]))[0]

### Saving/Loading Models

In [ ]:
def save_restore_model(model):
    # now you can save it to a file
    joblib_file = "joblib_model.pkl"  
    joblib.dump(model, joblib_file)

    # load from file
    joblib_model = joblib.load(joblib_file)

    # test loaded model
    print(make_prediction(model, 70, 65))
    print(make_prediction(model, 50, 70))
    print(make_prediction(model, 50, 65))

## Predictions using Support Vector Machine
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

References:
* <https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html>
* <https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code>
* <https://scikit-learn.org/stable/modules/svm.html#kernel-functions>

In [ ]:
%%time
# train C-Support Vector Classification (SVC) model and plot
model = SVC(kernel='linear', C=1, gamma='auto')
model.fit(X_train, y_train)
plot_model(model, 'SVC with linear kernel')

In [ ]:
# use linear SVC model
print(cross_validation(model))
print(make_prediction(model, 70, 65))
print(make_prediction(model, 50, 70))
print(make_prediction(model, 50, 65))
save_restore_model(model)

In [ ]:
%%time
# train C-Support Vector Classification (SVC) model and plot
model = SVC(kernel='rbf', C=1, gamma='auto')
model.fit(X_train, y_train)
plot_model(model, 'SVC with rbf kernel')

In [ ]:
# use rbf SVC model
print(cross_validation(model))
print(make_prediction(model, 70, 65))
print(make_prediction(model, 50, 70))
print(make_prediction(model, 50, 65))

## Predictions using K-Nearest Neighbors Classifier
* k-nearest neighbors algorithm (k-NN)
* <https://en.wikipedia.org/wiki/K-nearest_neighbor_algorithm>

In [ ]:
%%time
# train k-NN model and plot
model = KNeighborsClassifier(n_neighbors=2, algorithm='auto')
model.fit(X_train, y_train)
plot_model(model, 'k-NN Classifier')

In [ ]:
# use k-NN model
print(cross_validation(model))
print(make_prediction(model, 70, 65))
print(make_prediction(model, 50, 70))
print(make_prediction(model, 50, 65))

## Predictions using Random Forest Classifier
* Estimator that fits a number of decision tree classifiers on various sub-samples of the dataset
* Uses averaging to improve the predictive accuracy and control over-fitting
* <https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html>

In [ ]:
%%time
# train Random Forest Classifier model and plot
model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
model.fit(X_train, y_train)
plot_model(model, 'Random Forest Classifier')

In [ ]:
# use Random Forest Classifier model
print(cross_validation(model))
print(make_prediction(model, 70, 65))
print(make_prediction(model, 50, 70))
print(make_prediction(model, 50, 65))